In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

23/06/27 23:05:40 WARN Utils: Your hostname, Jasons-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.12 instead (on interface en0)
23/06/27 23:05:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/27 23:05:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# add the file to the sparkfiles context
spark.sparkContext.addFile(url)

# make the dataframe based on the candy dataset from URL
homesDF = spark.read.option('header', 'true').csv(
    SparkFiles.get("home_sales_revised.csv"),
    inferSchema=True,
    sep=','
)

homesDF.show(truncate=False)

+------------------------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|id                                  |date      |date_built|price |bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+------------------------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d6-9c31-7398aa8f6089|2022-04-08|2016      |936923|4       |3        |3167       |11733   |2     |1         |76  |
|7530a2d8-1ae3-4517-9f4a-befe060c4353|2021-06-13|2013      |379628|2       |2        |2235       |14384   |1     |0         |23  |
|43de979c-0bf0-4c9f-85ef-96dc27b258d5|2019-04-12|2014      |417866|2       |2        |2127       |10575   |2     |0         |0   |
|b672c137-b88c-48bf-9f18-d0a4ac62fb8b|2019-10-16|2016      |239895|2       |2        |1631       |11149   |2     |0         |0   |
|e0726d4d-d595-4074-8283-4139a54d0d63|2022-01-08|2017      |424418|3       |2      

In [4]:
# 2. Create a temporary view of the DataFrame.

homesDF.createOrReplaceTempView("home_sales")

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

queryString = """
(
    SELECT YEAR(date) AS year, 
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY YEAR(date)
)
"""
spark.sql(queryString).show(truncate=False)



+----+-------------+
|year|average_price|
+----+-------------+
|2019|300263.7     |
|2020|298353.78    |
|2021|301819.44    |
|2022|296363.88    |
+----+-------------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

queryString = """
(
    SELECT date_built AS year_built, 
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3
    GROUP BY date_built
    ORDER BY date_built
)
"""
spark.sql(queryString).show(truncate=False)



+----------+-------------+
|year_built|average_price|
+----------+-------------+
|2010      |297001.32    |
|2011      |300603.7     |
|2012      |298451.99    |
|2013      |302956.63    |
|2014      |302426.89    |
|2015      |299061.15    |
|2016      |302937.87    |
|2017      |300313.63    |
+----------+-------------+



In [9]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

queryString = """
(
    SELECT date_built AS year_built, 
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND (sqft_living + sqft_lot) >= 2000
    GROUP BY date_built
    ORDER BY date_built
)
"""
spark.sql(queryString).show(truncate=False)

+----------+-------------+
|year_built|average_price|
+----------+-------------+
|2010      |280447.23    |
|2011      |281413.45    |
|2012      |295858.68    |
|2013      |295142.13    |
|2014      |294195.13    |
|2015      |291788.36    |
|2016      |287812.6     |
|2017      |282026.59    |
+----------+-------------+



In [11]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT price, view FROM home_sales WHERE price >=350000").show()
spark.sql("SELECT ROUND(AVG(price), 2) AS average_price, ROUND(AVG(view), 2) AS average_view FROM home_sales WHERE price >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------+----+
| price|view|
+------+----+
|936923|  76|
|379628|  23|
|417866|   0|
|424418|   4|
|419199|   6|
|398667|   7|
|437958|  26|
|437375|  34|
|377485|  23|
|417215|  48|
|372151|  16|
|391574|  10|
|419543|  24|
|843191|  91|
|429183|  49|
|395178|  37|
|429030|  23|
|396508|  25|
|399016|  23|
|409011|   1|
+------+----+
only showing top 20 rows

+-------------+------------+
|average_price|average_view|
+-------------+------------+
|    473796.26|       32.26|
+-------------+------------+

--- 0.754871129989624 seconds ---


In [13]:
# 7. Cache the the temporary table homesView.
# use .spark.sql() to cache our table
spark.sql('cache table home_sales')

DataFrame[]

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [16]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT price, view FROM home_sales WHERE price >=350000").show()
spark.sql("SELECT ROUND(AVG(price), 2) AS average_price, ROUND(AVG(view), 2) AS average_view FROM home_sales WHERE price >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))


+------+----+
| price|view|
+------+----+
|936923|  76|
|379628|  23|
|417866|   0|
|424418|   4|
|419199|   6|
|398667|   7|
|437958|  26|
|437375|  34|
|377485|  23|
|417215|  48|
|372151|  16|
|391574|  10|
|419543|  24|
|843191|  91|
|429183|  49|
|395178|  37|
|429030|  23|
|396508|  25|
|399016|  23|
|409011|   1|
+------+----+
only showing top 20 rows

+-------------+------------+
|average_price|average_view|
+-------------+------------+
|    473796.26|       32.26|
+-------------+------------+

--- 0.7139039039611816 seconds ---


Uncached query time: 0.7549 sec <br>
Cached query time: 0.7139 sec

In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
homesDF.write.partitionBy("date_built").parquet('homesPartionData', mode='overwrite')


In [18]:
# 11. Read the formatted parquet data.
homesParquetDF = spark.read.parquet('homesPartionData')

In [19]:
# 12. Create a temporary table for the parquet data.
homesParquetDF.createOrReplaceTempView("homesParquetView")

In [21]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

spark.sql("SELECT price, view FROM homesParquetView WHERE price >=350000").show()
spark.sql("SELECT ROUND(AVG(price), 2) AS average_price, ROUND(AVG(view), 2) AS average_view FROM homesParquetView WHERE price >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------+----+
|  price|view|
+-------+----+
| 391574|  10|
| 419543|  24|
| 409011|   1|
| 425154|   4|
| 373139|  39|
| 797862|  90|
| 401792|  50|
| 352237|   6|
| 417244|   0|
| 425489|  26|
| 391519|  17|
| 443507|  10|
| 578348|  58|
|1256833|  82|
| 379777|   2|
| 383976|  44|
| 413683|  45|
| 697669|  88|
| 355690|  49|
| 813113|  56|
+-------+----+
only showing top 20 rows

+-------------+------------+
|average_price|average_view|
+-------------+------------+
|    473796.26|       32.26|
+-------------+------------+

--- 0.8584692478179932 seconds ---


Parquet partitioned query: 0.8585 sec

In [23]:
# 14. Uncache the homesView temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [24]:
# 15. Check if the homesView is no longer cached
spark.catalog.isCached('home_sales')


False